In [1]:
devtools::install_github("clairbarnes/rwwa")
library(rwwa)

Skipping install of 'rwwa' from a github remote, the SHA1 (a428e9e0) has not changed since last install.
  Use `force = TRUE` to force installation

Loading required package: extRemes

Loading required package: Lmoments

Loading required package: distillery


Attaching package: ‘extRemes’


The following objects are masked from ‘package:stats’:

    qqnorm, qqplot


Loading required package: plyr


Attaching package: ‘plyr’


The following object is masked from ‘package:distillery’:

    is.formula




# Linear trend per grid cell

## West Asia

In [13]:
gmst <- read.table("data/gmst.txt", col.names = c("year", "gmst"))
gmst$gmst <- gmst$gmst - gmst$gmst[gmst$year == 2024]

cov_2023 <- data.frame(gmst = gmst$gmst[gmst$year == 2024])
cov_cf <- rbind("hist" = cov_2023 - c(1.2, 0))

In [23]:
ds <- "era5"
fnm <- paste0("data/",ds,"_WA_flattened.csv")

df <- read.csv(fnm, skip = 3, header = F)
colnames(df)[1] <- "year"
df <- merge(gmst, df)

In [24]:
# fit nonstationary model to all grid cells
models <- sapply(colnames(df)[grepl("V[0-9]",colnames(df))], function(cnm) {
    df_c <- df[,c("year", "gmst", cnm)]
    tryCatch(fit_ns("gev", "shift", df_c, cnm, c("gmst"), lower = F), error = function(cond) {return(NULL)})
}, simplify = F)

In [25]:
fitted_res <- sapply(models, function(mdl) {
    if(is.null(unlist(mdl)[1])) { return(rep(NA, 10)) } else { mdl_ests(mdl, cov_f = cov_2023, cov_cf = cov_cf) } 
}, simplify = T)
fitted_res <- rbind(fitted_res)
write.csv(fitted_res, paste0("res/res_",ds, "_WA_gridded.csv"))

## East Asia

In [10]:
gmst <- read.table("data/gmst.txt", col.names = c("year", "gmst"))
gmst$gmst <- gmst$gmst - gmst$gmst[gmst$year == 2024]

nino <- read.csv("data/nino_djf.csv", col.names = c("year", "nino"))

cov_2023 <- data.frame(gmst = gmst$gmst[gmst$year == 2024], nino = nino$nino[nino$year == 2024])
cov_cf <- rbind("hist" = cov_2023 - c(1.2, 0), "neut" = c(cov_2023$gmst, 0))

In [ ]:
ds <- "era5"
fnm <- paste0("data/",ds,"_EA_flattened.csv")

df <- read.csv(fnm, skip = 3, header = F)
colnames(df)[1] <- "year"
df <- merge(merge(nino, gmst), df)

# fit nonstationary model to all grid cells
models <- sapply(colnames(df)[grepl("V[0-9]",colnames(df))], function(cnm) {
    df_c <- df[,c("year", "gmst", "nino", cnm)]
    tryCatch(fit_ns("gev", "shift", df_c, cnm, c("gmst", "nino"), lower = F), error = function(cond) {return(NULL)})
}, simplify = F)

In [ ]:
# get all model resuls & save as .csv to be reconstructed & plotted in python
fitted_res <- sapply(models, function(mdl) {
    if(is.null(unlist(mdl)[1])) { return(rep(NA, 14)) } else { mdl_ests(mdl, cov_f = cov_2023, cov_cf = cov_cf) } 
}, simplify = T)

write.csv(fitted_res, paste0("res/res_",ds, "_EA_gridded.csv"))